# PARAMETRIC ELLIPTIC OPTIMAL CONTROL

# Import di tutti i pacchetti

In [128]:
nome = 'nome' #Nome che voglio dare alla simulazione così da distinguere loss e plot per tutto

import argparse
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.nn import Softplus

#Questo serve a fare in modo che tutti i problemi abbiano come riferimento la stessa cartella di PINA
#così non si creano versioni diverse o sovrapposizioni
import sys
sys.path.append('C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO')

from pina import LabelTensor
from pina.solvers import PINN
from pina.model import MultiFeedForward
from pina.plotter import Plotter
from pina.trainer import Trainer
from pina.model import FeedForward
from pina.callbacks import MetricTracker

from pina import Condition
from pina.geometry import CartesianDomain
from pina.equation import SystemEquation, FixedValue
from pina.problem import SpatialProblem, ParametricProblem
from pina.operators import laplacian

# Definizione del problema
Il problema di base ha due parametri che sono 
- $\mu\in[0.5, 3]$ 
- $\alpha\in[0.01,1]$

In [129]:
# ===================================================== #                                        
#           u --> field variable                        #
#           p --> field variable                        #
#           y --> field variable                        #
#           x1, x2 --> spatial variables                #
#           mu, alpha --> problem parameters            #
#                                                       #
#           https://arxiv.org/pdf/2110.13530.pdf        #                         
# ===================================================== #


class ParametricEllipticOptimalControl(SpatialProblem, ParametricProblem):

    # setting spatial variables ranges
    xmin, xmax, ymin, ymax = -1, 1, -1, 1
    x_range = [xmin, xmax]
    y_range = [ymin, ymax]
    # setting parameters range
    amin, amax = 0.01, 1
    mumin, mumax = 0.5, 3
    mu_range = [mumin, mumax]
    a_range = [amin, amax]
    # setting field variables
    output_variables = ['u', 'y', 'p']
    # setting spatial and parameter domain
    spatial_domain = CartesianDomain({'x1': x_range, 'x2': y_range})
    parameter_domain = CartesianDomain({'mu': mu_range, 'alpha': a_range})

    # equation terms as in https://arxiv.org/pdf/2110.13530.pdf
    def term1(input_, output_):
        laplace_p = laplacian(output_, input_, components=['p'], d = ['x1', 'x2'])
        return output_.extract(['y']) - input_.extract(['mu']) - laplace_p

    def term2(input_, output_):
        laplace_y = laplacian(output_, input_, components=['y'], d = ['x1', 'x2'])
        return - laplace_y - output_.extract(['u'])
    
    def fixed_y(input_, output_):
        return output_.extract(['y'])

    # NOTA BENE, qui la condizione si trova su u e non su p perché p lo si calcola tramite il passo forward del PI-Arch,
    # la condizione si "trasferisce" naturalmente tramite il passo, non avrebbe senso porla qui su p perché tanto non me lo calcolo
    # proprio
    def fixed_u(input_, output_):
        return output_.extract(['u']) 

    # setting problem condition formulation
    conditions = {
        'gamma1': Condition(
            location=CartesianDomain({'x1': x_range, 'x2':  1, 'mu': mu_range, 'alpha': a_range}),
            equation=SystemEquation([fixed_y, fixed_u])),
        'gamma2': Condition(
            location=CartesianDomain({'x1': x_range, 'x2': -1, 'mu': mu_range, 'alpha': a_range}),
            equation=SystemEquation([fixed_y, fixed_u])),
        'gamma3': Condition(
            location=CartesianDomain({'x1':  1, 'x2': y_range, 'mu': mu_range, 'alpha': a_range}),
            equation=SystemEquation([fixed_y, fixed_u])),
        'gamma4': Condition(
            location=CartesianDomain({'x1': -1, 'x2': y_range, 'mu': mu_range, 'alpha': a_range}),
            equation=SystemEquation([fixed_y, fixed_u])),
        'D': Condition(location = CartesianDomain({'x1': x_range, 'x2': y_range, 'mu': mu_range, 'alpha': a_range}), equation=SystemEquation([term1, term2])),
    }

# DEFINIZIONE ARCHITETTURA DELLA PINN

In [130]:
#EXTRA FEATURE
class myFeature(torch.nn.Module):
    
    def __init__(self):
        super(myFeature, self).__init__()

    def forward(self, x):
        t = (-x.extract(['x1'])**2+1) * (-x.extract(['x2'])**2+1)
        return LabelTensor(t, ['k0'])


#FORWARD
class CustomMultiDFF(MultiFeedForward):

    def __init__(self, dff_dict):
        super().__init__(dff_dict)

    #Original forward
    def forward(self, x):
        out = self.uu(x)
        out.labels = ['u', 'y']
        p = LabelTensor((out.extract(['u']) * x.extract(['alpha'])), ['p'])
        
        return out.append(p)

In [131]:
if __name__ == "__main__":
    
    seed = 316680
    torch.manual_seed(seed)
    
    epochs = 1
    flag_extra_feature = 1
    
    parser = argparse.ArgumentParser(description = "Run PINA")
    parser.add_argument("--load", help = "directory to save or load file", type = str)
    parser.add_argument("--features", help = "extra features", type = int, default = flag_extra_feature)
    parser.add_argument("--epochs", help = "extra features", type = int, default = epochs)
    parser.add_argument('-f') #Serve per risolvere l'errore di sotto
    args = parser.parse_args()
    
    if args.features is None:
        args.features = 0
    
    # extra features
    feat = [myFeature()] if args.features else []
    args = parser.parse_args()
    
    # create problem and discretise domain
    opc = ParametricEllipticOptimalControl()
    opc.discretise_domain(n = 10, mode='grid', variables = ['x1', 'x2'], locations = ['D'])
    opc.discretise_domain(n = 10, mode='grid', variables = ['mu', 'alpha'], locations = ['D'])
    opc.discretise_domain(n = 10, mode='grid', variables = ['x1', 'x2'], locations = ['gamma1', 'gamma2', 'gamma3', 'gamma4'])
    opc.discretise_domain(n = 10, mode='grid', variables = ['mu', 'alpha'], locations = ['gamma1', 'gamma2', 'gamma3', 'gamma4'])

    # Questo comando serve per plottare i samples se dovesse servire
    # plt.scatter(opc.input_pts['D'].extract(['alpha']), opc.input_pts['D'].extract(['mu']))
    
    # Architettura
    model = CustomMultiDFF( 
        {
            'uu': {
                'input_dimensions': 4 + len(feat),  #due input spaziali più due parametri + l'extra feature
                'output_dimensions': 2,
                'layers': [40, 40, 20],
                'func': Softplus,
            },
        }
    )    
    
    # Creazione dell''stanza di PINN
    pinn = PINN(problem = opc, model = model, optimizer_kwargs = {'lr' : 0.002}, extra_features = feat)
    
    # Creazione di istanza di Trainer
    directory = 'pina.parametric_optimal_control_{}'.format(bool(args.features))
    trainer = Trainer(solver = pinn, accelerator='cpu', max_epochs = args.epochs, default_root_dir = directory)
    
    # callbacks = [MetricTracker()] questo si può usare solo se ho abbastanza spazio, è molto pesante, serve per plottare la loss

    #Training
    trainer.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type    | Params
----------------------------------------
0 | _loss       | MSELoss | 0     
1 | _neural_net | Network | 2.7 K 
----------------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)
C:\Users\Andrea\anaconda3\envs\PINA2\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.96it/s, v_num=37, gamma1_loss=0.0245, gamma2_loss=0.0251, gamma3_loss=0.0264, gamma4_loss=0.0234, D_loss=1.440, mean_loss=0.308]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.86it/s, v_num=37, gamma1_loss=0.0245, gamma2_loss=0.0251, gamma3_loss=0.0264, gamma4_loss=0.0234, D_loss=1.440, mean_loss=0.308]


# GRAFICI

In [127]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 3, 'alpha': 1}, components='u', filename = nome + '_u.pdf') 

In [7]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 3, 'alpha': 1}, components='y', filename = nome + '_y.pdf')

In [8]:
plotter = Plotter()
plotter.plot(pinn, fixed_variables={'mu': 3, 'alpha': 1}, components='p', filename = nome + '_p.pdf')

# Loss

In [ ]:
#Qui salvo la loss function
andamento_loss = trainer._model.lossVec
def salva_variabile(file, variabile):
    with open(file, 'w') as f:
        f.write(repr(variabile))

# Chiama la funzione per salvare la variabile
salva_variabile('loss_'+ nome +'.txt', andamento_loss) #Qui per salvare la loss

# Grafico loss
plt.loglog(andamento_loss)
plt.gcf().savefig(nome + 'grafico_loss.pdf', format='pdf') # Qui per salvare il grafico della loss

# Calcolo della norma L2
Per farlo estraggo prima i risultati da una simulazione FEM

In [112]:
n = 638
fem_u = torch.tensor(np.load("C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO/Codici_notebook/Parametric_elliptic/FEM_Elliptic/alpha_1/control.npy"), dtype=torch.float).view(-1, 1)
fem_p = torch.tensor(np.load("C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO/Codici_notebook/Parametric_elliptic/FEM_Elliptic/alpha_1/adjoint.npy"), dtype=torch.float).view(-1, 1)
fem_y = torch.tensor(np.load("C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO/Codici_notebook/Parametric_elliptic/FEM_Elliptic/alpha_1/state.npy"), dtype=torch.float).view(-1, 1)
fem_x = torch.tensor(np.load("C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO/Codici_notebook/Parametric_elliptic/FEM_Elliptic/alpha_1/x.npy"), dtype=torch.float).view(-1, 1)                                
fem_y = torch.tensor(np.load("C:/Users/Andrea/Desktop/Poli/Tesi magistrale/reporitory_SISSA_PoliTO/Codici_notebook/Parametric_elliptic/FEM_Elliptic/alpha_1/y.npy"), dtype=torch.float).view(-1, 1)


In [119]:
x_labelT = LabelTensor(fem_x, ['x1'])
y_labelT = LabelTensor(fem_y, ['x2'])
mu_labelT = LabelTensor(torch.full((n,), 3, dtype=torch.float).view(-1, 1), ['mu']) 
alfa_labelT = LabelTensor(torch.full((n,), 1, dtype=torch.float).view(-1, 1), ['alpha'])
input = x_labelT.append(y_labelT).append(mu_labelT).append(alfa_labelT)

output = pinn.forward(input)
output.labels = ['u', 'y', 'p']

errore_u = (output.extract(['u']) - fem_u).reshape(n,)
errore_y = (output.extract(['y']) - fem_y).reshape(n,)
errore_p = (output.extract(['p']) - fem_p).reshape(n,)

norma_errore_u = torch.dot(errore_u, errore_u).item()
norma_errore_y = torch.dot(errore_y, errore_y).item()
norma_errore_p = torch.dot(errore_p, errore_p).item()

with open(nome + 'l2_errors.txt', 'w') as file:
    file.write(f"norma_errore_u = {norma_errore_u}\n")
    file.write(f"norma_errore_y = {norma_errore_y}\n")
    file.write(f"norma_errore_p = {norma_errore_p}\n")